In [ ]:
import pandas as pd
import requests
import aiohttp
import asyncio
import json
import os
import sys
import math
from aiohttp import ClientSession
from urllib.error import HTTPError

In [ ]:
API = "https://api.elections.org.za"
token = ''
headers = {"Authorization": "Bearer " + token}

In [ ]:
ElectoralEventID = '402'
WardResults = []
MuniSeatResults = []

In [ ]:
wards_df = pd.read_csv('Wards.csv')
Wards = wards_df.values.tolist()
munis_df = pd.read_csv('Munis.csv')
Munis = munis_df.values.tolist()

In [ ]:
async def get_data(endpoint, url, query, session):
    api_url = API + url + str(query)
    try:
        response = await session.request(method='GET', url=api_url, headers=headers)
        response.raise_for_status()
    except HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"An error ocurred: {err}")
    response_json = await response.json()
    return response_json

In [ ]:
async def run_program(endpoint, url, query, session):
    try:
        response = await get_data(endpoint, url, query, session)
        
        ##### 
        ## LGEBallotResults-Ward
        #####
        
        if(endpoint == 'LGEBallotResults-Ward'):
            print(str(response['Province']) + ' - ' + str(response['Municipality']) + ' - ' + str(response['WardID']))

            for result in response['PartyBallotResults']:
                WardResults.append(
                    {
                        'ProvinceID': response['ProvinceID'],
                        'Province': response['Province'],
                        'Municipality': response['Municipality'],
                        'MunicipalityID': response['MunicipalityID'],
                        'WardID': response['WardID'],
                        'Ward_RegisteredVoters':response['RegisteredVoters'],
                        'Ward_SpoiltVotes':response['SpoiltVotes'],
                        'Ward_SpecialVotes':response['SpecialVotes'],
                        'Ward_PercVoterTurnout':response['PercVoterTurnout'],
                        'Ward_TotalVotesCast':response['TotalVotesCast'],
                        'Ward_TotalValidVotes':response['TotalValidVotes'],
                        'Ward_VDCount':response['VDCount'],
                        'Ward_VDWithResultsCaptured':response['VDWithResultsCaptured'],
                        
                        'PartyID': result['ID'],
                        'PartyName': result['Name'],
                        'Ward_ValidVotes': result['Ward_ValidVotes'],
                        'PR_ValidVotes': result['PR_ValidVotes'],
                        'DC40Perc_ValidVotes': result['DC40Perc_ValidVotes'],
                        'TotalValidVotes': result['TotalValidVotes'],
                        'PercOfVotes': result['PercOfVotes'],
                        'IsOnNational': result['IsOnNational']
                    }
                )
                
        ##### 
        ## LGESeatCalculationResults-Muni
        #####
                
        if(endpoint == 'LGESeatCalculationResults-Muni'):
            print(str(response['Province']) + ' - ' + str(response['Municipality']))
            
            for result in response['PartyResults']:
                MuniSeatResults.append(
                    {
                        'Muni_ProvinceID': response['ProvinceID'],
                        'Muni_Province': response['Province'],
                        'Muni_Municipality': response['Municipality'],
                        'Muni_MunicipalityID': response['MunicipalityID'],
                        'Muni_Quota': response['Quota'],
                        'Muni_TotalSeatsAvailable': response['TotalSeatsAvailable'],
                        'Muni_IndependentSeatsWon': response['IndependentSeatsWon'],
                        'Muni_WardCouncillorsWithNoPR': response['WardCouncillorsWithNoPR'],
                        'Muni_TotalValidVotes': response['TotalValidVotes'],
                        
                        'PartyID': result['ID'],
                        'PartyName': result['Name'],
                        'TotalValidVotes': result['TotalValidVotes'],
                        'TotalPartySeats': result['TotalPartySeats'],
                        'WardSeats': result['WardSeats'],
                        'PRSeats': result['PRSeats']
                    }
                )
        
        ##### 
        ## LGESeatCalculationResults-Muni-Simple
        #####
                
        if(endpoint == 'LGESeatCalculationResults-Muni-Simple'):
            print(str(response['Province']) + ' - ' + str(response['Municipality']))
            
            for result in response['PartyResults']:
                MuniSeatResults.append(
                    {
                        'Province': response['Province'],
                        'Geography': response['Municipality'].split(" ", 1)[0],
                        'PartyName': result['Name'],
                        'SeatType': 'Ward',
                        'Count': result['WardSeats'],
                    }
                )
                MuniSeatResults.append(
                    {
                        'Province': response['Province'],
                        'Geography': response['Municipality'].split(" ", 1)[0],
                        'PartyName': result['Name'],
                        'SeatType': 'PR',
                        'Count': result['PRSeats'],
                    }
                )
                
        ##### 
        ## LGESeatCalculationResults-Hung
        #####
                
        if(endpoint == 'LGESeatCalculationResults-Hung'):
#             print(str(response['Province']) + ' - ' + str(response['MunicipalityID']))
            
            muniResults = []
            
            
            for result in response['PartyResults']:
                muniResults.append(
                     {
                        'PartyName': result['Name'],
                        'TotalPartySeats': result['TotalPartySeats']
                     }
                 )
            
#             if(response['TotalSeatsAvailable']/2 > max(muniResults, key=lambda x:x['TotalPartySeats'])['TotalPartySeats']):
#                 is_hung = True
#                 majorityParty = ''
#                 short = math.ceil((response['TotalSeatsAvailable']/2 + 1) - max(muniResults, key=lambda x:x['TotalPartySeats'])['TotalPartySeats'])
#                 lead = max(muniResults, key=lambda x:x['TotalPartySeats'])['TotalPartySeats']
#             else:
#                 is_hung = False
#                 majorityParty = max(muniResults, key=lambda x:x['TotalPartySeats'])['PartyName']
#                 short = ''
#                 lead = ''
            
#             MuniSeatResults.append(
#                 {
#                     'Province': response['Province'],
#                     'Geography': response['Municipality'].split(" ", 1)[0],
#                     'MunicipalityID': response['MunicipalityID'],
#                     'isHung': is_hung,
#                     'TotalSeatsAvailable': response['TotalSeatsAvailable'],
#                     'majorityParty': majorityParty,
#                     'lead': lead,
#                     'short': short
                    
#                 }
#             )
            if(response['TotalSeatsAvailable']/2 > max(muniResults, key=lambda x:x['TotalPartySeats'])['TotalPartySeats']):
                majority_result = 'Hung'
            else:
                majority_result = 'Outright Majority'
                
            MuniSeatResults.append(
                {
                    'Province': response['Province'],
                    'Geography': response['Municipality'].split(" ", 1)[0],
                    'majority_result': majority_result
                    
                }
            )

            
    except Exception as err:
        print(f"Exception occured: {err}")
        pass

In [ ]:
# async with ClientSession() as session:
#     await asyncio.gather(*[run_program('LGEBallotResults-Ward','/api/v1/LGEBallotResults?ElectoralEventID=' + str(ElectoralEventID), '&ProvinceID=' + str(ward[0]) + '&MunicipalityID=' + str(ward[1]) + '&WardID=' + str(ward[2]), session) for ward in Wards])
    
#     print('Saving to CSV...')
#     wardResults_df = pd.DataFrame(WardResults) 
#     wardResults_df.to_csv('WardResults.csv', index=False)
#     print('Done')

# async with ClientSession() as session:
#     await asyncio.gather(*[run_program('LGESeatCalculationResults-Muni','/api/v1/LGESeatCalculationResults?ElectoralEventID=' + str(ElectoralEventID), '&MunicipalityID=' + str(muni[1]), session) for muni in Munis])
    
#     print('Saving to CSV...')
#     muniResults_df = pd.DataFrame(MuniSeatResults) 
#     muniResults_df.to_csv('MuniSeatResults.csv', index=False)
#     print('Done')
    
# async with ClientSession() as session:
#     await asyncio.gather(*[run_program('LGESeatCalculationResults-Muni-Simple','/api/v1/LGESeatCalculationResults?ElectoralEventID=' + str(ElectoralEventID), '&MunicipalityID=' + str(muni[1]), session) for muni in Munis])
    
#     print('Saving to CSV...')
#     muniResults_df = pd.DataFrame(MuniSeatResults) 
#     muniResults_df.to_csv('MuniSeatResultsSimple.csv', index=False)
#     print('Done')



# async with ClientSession() as session:
#     await asyncio.gather(*[run_program('LGESeatCalculationResults-Hung','/api/v1/LGESeatCalculationResults?ElectoralEventID=' + str(ElectoralEventID), '&MunicipalityID=' + str(muni[1]), session) for muni in Munis])
    
#     print('Saving to CSV...')
#     muniResults_df = pd.DataFrame(MuniSeatResults) 
#     muniResults_df.to_csv('MunisHung.csv', index=False)
#     print('Done')
    
#     print('Calculating total councils by party...')
#     muniResults_df = muniResults_df.loc[muniResults_df['isHung'] == False]
#     provinces = muniResults_df['Province'].unique()
    
#     print('Saving to CSV...')
#     province_df = muniResults_df.rename(columns={"majorityParty": "Party"}).groupby('Province')['Party'].value_counts().reset_index(name='Councils')
#     province_df.to_csv('PartyCouncilsByProvince.csv',index=False)
#     print('Done')



async with ClientSession() as session:
    await asyncio.gather(*[run_program('LGESeatCalculationResults-Hung','/api/v1/LGESeatCalculationResults?ElectoralEventID=' + str(ElectoralEventID), '&MunicipalityID=' + str(muni[1]), session) for muni in Munis])
    
#     print('Saving to CSV...')
    muniResults_df = pd.DataFrame(MuniSeatResults) 
#     muniResults_df.to_csv('MunisHung.csv', index=False)
#     print('Done')

In [ ]:
muniResults_df = pd.DataFrame(MuniSeatResults)
work_df = muniResults_df
work_df2 = work_df.groupby('Province')['majority_result'].value_counts().reset_index(name='count')
work_df2
work_df2.to_csv('MunisHungByProvince.csv', index=False)